# Segmenting and Clustering Neighborhoods in the city of Toronto, Canada

First we download all the dependencies that we will need.

In [7]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.2 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.21.0-py_0

The following packages will be UPDATED:

  openssl                                 1.1.1f-h516909a_0 --> 1.1.1g-h516909a_0



openssl-1.1.1g       | 2.1 MB    | ############

In [9]:
# installing beautiful soup package
!pip install BeautifulSoup4

     |████████████████████████████████| 112kB 13.2MB/s eta 0:00:01


In [10]:
import requests 
from bs4 import BeautifulSoup 


In [11]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
rpage = requests.get(URL) # get the requested url
data = rpage.text  # get the text data from the respose object
soup = BeautifulSoup(data)
# print(soup.prettify()) 
print('Page Scrapped.')

Page Scrapped.


In [12]:
table = soup.find('table')
table

<table class="wikitable sortable">
<tbody><tr>
<th>Postal code
</th>
<th>Borough
</th>
<th>Neighborhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park / Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor / Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park / Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern / Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3B
</td>
<td>North York
</td>
<td>Don Mills
</td></tr>
<tr>
<td>M4B
</td>
<td>Ea

In [13]:

# Get the data from each table row
wiki_table = soup.find('table')
table_row = wiki_table.find_all('tr')
dataframe = []
for row in table_row:
    table_data = row.find_all('td')
    data = [i.text.rstrip() for i in table_data]
#     print(data)
    dataframe.append(data)

dataframe 
del dataframe[0]
dataframe

[['M1A', 'Not assigned', ''],
 ['M2A', 'Not assigned', ''],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Regent Park / Harbourfront'],
 ['M6A', 'North York', 'Lawrence Manor / Lawrence Heights'],
 ['M7A', 'Downtown Toronto', "Queen's Park / Ontario Provincial Government"],
 ['M8A', 'Not assigned', ''],
 ['M9A', 'Etobicoke', 'Islington Avenue'],
 ['M1B', 'Scarborough', 'Malvern / Rouge'],
 ['M2B', 'Not assigned', ''],
 ['M3B', 'North York', 'Don Mills'],
 ['M4B', 'East York', 'Parkview Hill / Woodbine Gardens'],
 ['M5B', 'Downtown Toronto', 'Garden District / Ryerson'],
 ['M6B', 'North York', 'Glencairn'],
 ['M7B', 'Not assigned', ''],
 ['M8B', 'Not assigned', ''],
 ['M9B',
  'Etobicoke',
  'West Deane Park / Princess Gardens / Martin Grove / Islington / Cloverdale'],
 ['M1C', 'Scarborough', 'Rouge Hill / Port Union / Highland Creek'],
 ['M2C', 'Not assigned', ''],
 ['M3C', 'North York', 'Don Mills'],
 ['M4C', 'East York'

In [14]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 

# Create pandas dataframe and load it with data
neighs_df = pd.DataFrame.from_records(dataframe, columns=column_names)
# Remove the data cells with 'Not assigned' value in Borough
neighs_df = neighs_df[neighs_df.Borough != 'Not assigned']
# Replace the 'Not assigned' in Neighborhood with the respective Borough values.
neighs_df.Neighborhood = neighs_df.Neighborhood.replace('Not assigned', neighs_df.Borough)
neighs_df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [15]:
# Group Neighborhoods with same Postalcodes
neighs_df.shape

(103, 3)

In [16]:
# Geographical Coordinates
df1=pd.read_csv("https://cocl.us/Geospatial_data")
# Change the Postal Code to Postcode
df1.rename(columns={'Postal Code':'PostalCode'},inplace=True)
#Cancatenation
frames=[neighs_df,df1]
frames=pd.concat(frames, axis=1, sort=False)

In [17]:
# Merging the two columns on 'Postcode'
merge_columns=pd.merge(neighs_df, df1, left_on='PostalCode', right_on='PostalCode')
# Save the Data Frame
merge_columns.to_csv('neigbors_geographical.csv')
merge_columns.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


In [18]:
# find and match strings with only Toronto
toronto_data = merge_columns[merge_columns['Borough'].str.contains('Toronto', regex=False)]

In [19]:
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
9,M5B,Downtown Toronto,Garden District / Ryerson,43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


## CREATING A MAP OF TORONTO WITH THE NEIGHBORHOODS SUPERIMPOSED ON IT

In [22]:
# creating a map of Toronto using longitude and latitude values
map_toronto = folium.Map(location=[43.651070,-79.347015],zoom_start=10)
# add markers to map
for lat,lng,borough,neighbourhood in zip(toronto_data['Latitude'],toronto_data['Longitude'],toronto_data['Borough'],toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto

# Cluster Neighborhoods

In [28]:
# set number of clusters
k=5
toronto_cluster = toronto_data.drop(['PostalCode','Borough','Neighborhood'],1)
# run k-means clustering
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_cluster)
# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 2, 0, 0, 3, 0, 4, 0, 3, 2, 0, 3, 2, 0, 2, 1, 1, 1, 1,
       4, 1, 3, 4, 1, 3, 4, 1, 3, 1, 0, 0, 0, 0, 0, 0, 2], dtype=int32)

In [30]:
# inserting cluster labels into the dataframe
toronto_data.head()

,Cluster Labels,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,0,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
4,0,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
9,0,M5B,Downtown Toronto,Garden District / Ryerson,43.657162,-79.378937
15,0,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,4,M4E,East Toronto,The Beaches,43.676357,-79.293031


Visualizing the resulting clusters

In [32]:
# create map
map_clusters = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood'], toronto_data['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters